In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
from tqdm.notebook import tqdm, trange

"""Change to the data folder"""
new_path = "./new_train/new_train"

cuda_status = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 64

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, out, scene_ids, track_ids, agent_ids]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, scene_ids, track_ids, agent_ids]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(4, 8, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    #torch.nn.Conv2d(8, 24, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    #torch.nn.Linear(24 * 5 * 5, 512),
    #torch.nn.ReLU(),
    #torch.nn.Linear(512, 256),
    #torch.nn.ReLU(),
    #torch.nn.Linear(256, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [4]:
class LSTM(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(76, hidden_size=98, batch_first=True)
        self.linear = torch.nn.Linear(19*98, 7200)
        self.linear2 = torch.nn.Linear(5880, 6500)
        self.relu = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(6500, 6500)
        self.linear4 = torch.nn.Linear(6500, 7200)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = torch.flatten(out, start_dim=1)
        #predict = self.linear(out)
        #predict = torch.reshape(predict, torch.Size([64, 60, 30, 4]))
        out = self.relu(self.linear2(out))
        out = self.relu(self.linear3(out))
        out = self.relu(self.linear4(out))
        return out

model = LSTM()
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
model = torch.load('./models/1epochAdam.pt')
model.train()
model.to(device)
if cuda_status:
    model = model.cuda()

### Visualize the batch of sequences

In [5]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 5

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out, scene_ids, track_ids, agent_ids = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        x = torch.flatten(inp, start_dim=2)

        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None

        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        print(y_pred.shape)
        y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss))

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


torch.Size([64, 7200])
tensor(594.0446, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(773.6462, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(625.7133, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(587.3246, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(513.8085, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(436.3566, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(339.0095, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(386.2603, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(452.2932, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(415.5906, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(417.1761, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tensor(400.2621, device='cuda:0', grad_fn=<SqrtBackward>)
torch.Size([64, 7200])
tenso

KeyboardInterrupt: 

In [5]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 10
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
#learning_rate =0.01
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        #x = torch.flatten(inp, start_dim=2)
        x = inp.permute(0,2,1,3).float()
        x = torch.flatten(x, start_dim=2)
        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None
        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        #y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss))

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


tensor(594.0455, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(582.7543, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(609.7330, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(568.3273, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(541.2189, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(555.7875, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(469.7024, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(506.3892, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(560.7409, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(488.1715, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(532.1202, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(473.8800, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(518.3258, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(490.6623, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(534.3001, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(481.9059, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(451.9143, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(486.080

tensor(336.0473, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(354.9907, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(380.4819, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(357.7336, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(350.3830, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(376.3637, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(384.5926, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(319.9966, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(370.5426, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(347.5435, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(376.8003, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(374.1161, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(380.8490, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(360.7283, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(353.5679, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(368.7570, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(326.6108, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(359.726

tensor(362.0633, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(316.6688, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(315.6803, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(312.3532, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(328.7986, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(350.5054, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(326.9573, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(370.3326, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(353.0373, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(305.0288, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(329.2953, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(304.2745, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(308.2319, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(291.5103, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(321.1529, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(293.5720, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(317.5540, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(301.824

tensor(291.2493, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(331.6094, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(329.8109, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(288.3600, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(282.1042, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(276.5237, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(280.6007, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(277.5572, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(297.9434, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(340.4150, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(294.2826, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(311.3357, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(327.3871, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(281.6772, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(320.6797, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(284.9641, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(316.1655, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(339.388

tensor(275.4691, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(268.5952, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(322.5189, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(293.9802, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(263.1868, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(265.8124, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(292.6396, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(305.0789, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(292.8617, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(258.1378, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(264.7263, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(287.2624, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(265.0049, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(288.2513, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(284.8132, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(272.3416, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(293.3407, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(253.563

...

tensor(50.2834, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(74.2395, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(87.9733, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(69.8318, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(61.5134, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(76.0168, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(82.5956, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(66.1215, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(69.0368, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(66.5942, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(97.8607, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(65.2238, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(65.7766, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(57.0676, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(87.9336, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(61.1551, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(59.3934, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(80.5484, device='cuda:0'

tensor(61.1186, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(54.5123, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(69.9261, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(74.2635, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(72.5155, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(60.0093, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.9694, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.1628, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(57.2144, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(51.4019, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(67.8056, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(64.3920, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(85.3435, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(58.8927, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(80.6438, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(57.3516, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(84.8026, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(79.6020, device='cuda:0'

tensor(81.5099, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(86.3696, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(68.4151, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(58.6880, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(110.2016, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(68.4360, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(62.9809, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(66.6316, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(61.2803, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(69.9682, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(72.2684, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(65.6369, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(56.7608, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.9050, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(61.8308, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(57.0762, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(73.6634, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(64.5941, device='cuda:0

tensor(84.4900, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(61.9565, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(64.4742, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(72.8646, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(79.1665, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.3630, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(96.5554, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(86.4178, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(59.3597, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(86.5144, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(76.3858, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(69.2283, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(62.0335, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(77.5253, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(89.9757, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(91.4819, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(64.2619, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.9502, device='cuda:0'

tensor(58.2388, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(143.0792, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(80.3361, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(59.7471, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(67.3141, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(81.5150, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(77.9833, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(69.8646, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(74.6027, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(54.9339, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(61.8683, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(73.0841, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(78.8971, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(77.3279, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.4703, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(71.5087, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(73.9581, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(86.1863, device='cuda:0

In [6]:
torch.save(model, './models/10epochLSTMRSM.pt')

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
model = torch.load('./models/6epochmodel.pt')
model.eval()
model.to(device)
if cuda_status:
    model = model.cuda()

In [7]:
import pandas as pd

# Submission output
writeCSV = True
val_path = "./new_val_in/new_val_in"

if writeCSV:
    
    dataset = ArgoverseDataset(data_path=val_path)
    test_loader = DataLoader(dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)
    
    data = []
    
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(tqdm(test_loader)):
            inp, scene_ids, track_ids, agent_ids = sample_batch
            inp = torch.flatten(inp, start_dim=2)

            if cuda_status:
                model = model.cuda()
                x = inp.cuda()
            else:
                x = inp

            y_pred = None

            # Forward pass: predict y by passing x to the model.    
            y_pred = model(x.float())
            y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))
            
            for i in range(batch_sz):
                row = []
                row.append(scene_ids[i].item())
                curr = y_pred[i]
                
                agent_id = agent_ids[i]
                
                for j in range(30):
                    vehicle_index = 0
                    found = False
                    while not found:
                        if track_ids[i][vehicle_index][j][0] == agent_id:
                            found = True
                        else:
                            vehicle_index += 1

                    row.append(str(curr[vehicle_index][j][0].item()))
                    row.append(str(curr[vehicle_index][j][1].item()))
                    
                data.append(row)

    df = pd.DataFrame(data, columns = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35','v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52','v53','v54','v55','v56','v57','v58','v59','v60'])
    print(df)
    df.to_csv('submission.csv', index=False)
                
                
                

RuntimeError: input.size(-1) must be equal to input_size. Expected 240, got 76